In [552]:
import pandas as pd
import re
import requests as req
import os
from dotenv import load_dotenv
load_dotenv()
from pymongo import MongoClient
import folium
from folium import plugins
from folium.plugins import MiniMap


api_key = os.environ["KEY"] 

In [553]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [554]:
df = db.companies

## Query Pymongo eligiendo empresas tecnológicas, con información sobre oficinas, con empleados, fundadas a partir de 2005 y que no hayan quebrado

In [555]:
companies_with_coord = pd.DataFrame(df.find(
    {'$and': 
     [
         {'$or':
          [
         {'category_code':'games_video'},
         {'category_code':'web'},
         {'category_code':'software'},
         {'category_code':'mobile'},
         {'category_code':'advertising'},
         {'category_code':'ecommerce'},
         {'category_code':'design'},
         {'category_code':'biotech'},
         {'category_code':'cleantech'},
         {'category_code':'travel'}
     ]
         },
         {"number_of_employees": 
          {'$not': 
           {'$eq': None}
          }
         },
         {"offices.latitude": 
          {'$exists': True}
         },
          {"founded_year": 
          {'$gte': 2005}
         },
         {"offices.longitude": 
          {'$exists': True}
         },
         {"offices.latitude": 
          {'$not': 
           {'$eq': None}
          }
         },
         {"offices.longitude": 
          {'$not': 
           {'$eq': None}
          }
         },
         {'deadpooled_year': 
             {'$eq' :None}
         }
     ]
    }))

In [556]:
companies_with_coord.shape

(2102, 42)

In [557]:
companies_with_coord['founded_year'].describe()

count    2102.000000
mean     2007.091342
std         1.204557
min      2005.000000
25%      2006.000000
50%      2007.000000
75%      2008.000000
max      2013.000000
Name: founded_year, dtype: float64

## Limpiando campo total_money_raised

In [558]:
letters=[]
for i in companies_with_coord['total_money_raised']:
    if re.findall('[^0-9.,]+',i) not in letters:
        letters.append(re.findall('[^0-9.,]+',i))
    

In [559]:
letters

[['$', 'M'],
 ['$'],
 ['$', 'k'],
 ['€', 'M'],
 ['€', 'k'],
 ['£', 'k'],
 ['£', 'M'],
 ['C$', 'k']]

In [560]:
dict_coin={
    '$' : 'USD',
    '£' : 'GBP',
    'C$' : 'CAD',
    'kr' : 'SEK',
    '€' : 'EUR'
}
coins= ['USD','GBP','CAD','SEK']

In [561]:
coindict={}
for i in coins:
    res = req.get('https://api.exchangeratesapi.io/latest')
    data = res.json()
    coindict[i] = data['rates'][i]

In [562]:
coindict

{'USD': 1.1215, 'GBP': 0.9034, 'CAD': 1.4654, 'SEK': 10.5168}

In [563]:
dict_quantity={
    '$': 1,
    'M': 1000000, 
    'B':1000000000, 
    'k': 1000, 
    '£':coindict['GBP']/coindict['USD'],  
    'C$':coindict['CAD']/coindict['USD'], 
    'kr':coindict['SEK']/coindict['USD'],
    '€':1/coindict['USD']
}

In [564]:
total_money_raised_normalized=[]
for i in companies_with_coord['total_money_raised']:
    x = float(re.findall('[0-9.]+',str(i))[0])
    y=1
    for j in re.findall('[^0-9.]+',str(i)):
        y = y* dict_quantity[j]
    total_money_raised_normalized.append(x*y)

In [565]:
len(total_money_raised_normalized)

2102

In [566]:
companies_with_coord['total_money_raised_normalized_USD']= total_money_raised_normalized

In [567]:
companies_with_coord['total_money_raised_normalized_USD'].head()

0    16500000.0
1    45000000.0
2    21000000.0
3    23400000.0
4    88700000.0
Name: total_money_raised_normalized_USD, dtype: float64

In [568]:
#companies_first_filter_2 = companies_first_filter[companies_first_filter['total_money_raised_normalized_USD']>=1000000]

In [569]:
#companies_first_filter_2.head()

## Duplicando registros de empresas con varias oficinas, creando un registro por cada oficina

In [570]:
companies_with_coord_intermediate=companies_with_coord.drop(columns='offices')

In [571]:
office_list=[]
for i in companies_with_coord['offices']:
    if not isinstance(i,list):
        office_list.append(list(i))
    else: office_list.append(i)

In [572]:
#companies_first_filter_3 = companies_first_filter_2.copy()
companies_with_coord['offices']=office_list

In [573]:
companies_with_coord['offices'].head()

0    [{'description': 'Headquarters', 'address1': '...
1    [{'description': '', 'address1': '100 5th Ave ...
2    [{'description': '', 'address1': '3525 Eastham...
3    [{'description': None, 'address1': '442 Post S...
4    [{'description': '', 'address1': '475 Park Ave...
Name: offices, dtype: object

In [574]:
companies_with_coord2 = pd.DataFrame(companies_with_coord.offices.tolist()).stack().reset_index().drop(columns='level_1')
companies_with_coord2.columns=['index','office']

In [575]:
display(companies_with_coord2.head())

,index,office
0,0,"{'description': 'Headquarters', 'address1': '9..."
1,1,"{'description': '', 'address1': '100 5th Ave F..."
2,2,"{'description': '', 'address1': '3525 Eastham ..."
3,3,"{'description': None, 'address1': '442 Post St..."
4,4,"{'description': '', 'address1': '475 Park Ave ..."


In [576]:
companies_with_coord.reset_index(inplace=True)

In [577]:
companies_with_coord.head()

,index,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,...,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD
0,0,52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,...,"[{'name': 'Geni', 'permalink': 'geni'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0
1,1,52cdef7c4bab8bd675297d9d,"{'price_amount': None, 'price_currency_code': ...",[],None,http://blog.joost.com/atom.xml,http://blog.joost.com,games_video,"[{'competitor': {'name': 'Babelgum', 'permalin...",Sat Jun 09 07:05:30 UTC 2007,...,"[{'name': 'Joost', 'permalink': 'joost'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"iptv, babelgum, television, video, thevenicepr...",$45M,None,Fri Mar 15 23:31:11 UTC 2013,"[{'embed_code': '<object width=""425"" height=""3...",45000000.0
2,2,52cdef7c4bab8bd675297da9,None,[],,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,"[{'competitor': {'name': 'Topicle', 'permalink...",Thu Jun 14 03:42:20 UTC 2007,...,"[{'name': 'Mahalo', 'permalink': 'mahalo'}]",[],"[{'is_past': False, 'title': 'Founder, CEO', '...",[],"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",21000000.0
3,3,52cdef7c4bab8bd675297dab,"{'price_amount': None, 'price_currency_code': ...",[],,http://feeds.feedburner.com/kyte_blog,http://kyte.com/blog,games_video,"[{'competitor': {'name': 'Ustream', 'permalink...",Thu Jun 14 18:26:11 UTC 2007,...,"[{'name': 'Kyte', 'permalink': 'kyte'}, {'name...",[],"[{'is_past': False, 'title': 'CTO and co-found...",[],"video, mobile, iphone-app, video-platform, mob...",$23.4M,kyte,Mon Oct 28 09:34:37 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",23400000.0
4,4,52cdef7c4bab8bd675297daf,"{'price_amount': 62500000, 'price_currency_cod...",[],,,,mobile,[],Sat Jun 16 05:53:38 UTC 2007,...,"[{'name': 'MyFree411', 'permalink': 'myfree411'}]","[{'title': 'Public Relations', 'is_past': Fals...","[{'is_past': False, 'title': 'Founder and CTO'...",[],"directoryassistance, advertising, mobile",$88.7M,Marchex,Mon Jun 03 05:26:22 UTC 2013,[],88700000.0


In [578]:
companies_with_coord_offices= companies_with_coord2.merge(companies_with_coord, how='left', on='index')

In [579]:
companies_with_coord_offices.columns

Index(['index', 'office', '_id', 'acquisition', 'acquisitions', 'alias_list',
       'blog_feed_url', 'blog_url', 'category_code', 'competitions',
       'created_at', 'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds',
       'total_money_raised_normalized_USD'],
      dtype='object')

In [580]:
def getFirst(data):
    data = data['office']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data['latitude'] and data['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data['longitude'], data['latitude']]
        }

    return {
        "lat": data['latitude'],
        "lng": data['longitude'],
        "oficina_principal": principal
    }

In [581]:
companies_with_coord_offices_sub = companies_with_coord_offices[["office"]].apply(getFirst, result_type="expand", axis=1)

In [582]:
companies_with_coord_offices_sub.head()

,lat,lng,oficina_principal
0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
2,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,..."
4,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,..."


In [583]:
companies_with_coord_offices_clean = pd.concat([companies_with_coord_offices,companies_with_coord_offices_sub], axis=1)

In [584]:
companies_with_coord_offices_clean.head()

,index,office,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,...,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD,lat,lng,oficina_principal
0,0,"{'description': 'Headquarters', 'address1': '9...",52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",...,[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,1,"{'description': '', 'address1': '100 5th Ave F...",52cdef7c4bab8bd675297d9d,"{'price_amount': None, 'price_currency_code': ...",[],None,http://blog.joost.com/atom.xml,http://blog.joost.com,games_video,"[{'competitor': {'name': 'Babelgum', 'permalin...",...,[],"iptv, babelgum, television, video, thevenicepr...",$45M,None,Fri Mar 15 23:31:11 UTC 2013,"[{'embed_code': '<object width=""425"" height=""3...",45000000.0,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
2,2,"{'description': '', 'address1': '3525 Eastham ...",52cdef7c4bab8bd675297da9,None,[],,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,"[{'competitor': {'name': 'Topicle', 'permalink...",...,[],"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",21000000.0,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,3,"{'description': None, 'address1': '442 Post St...",52cdef7c4bab8bd675297dab,"{'price_amount': None, 'price_currency_code': ...",[],,http://feeds.feedburner.com/kyte_blog,http://kyte.com/blog,games_video,"[{'competitor': {'name': 'Ustream', 'permalink...",...,[],"video, mobile, iphone-app, video-platform, mob...",$23.4M,kyte,Mon Oct 28 09:34:37 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",23400000.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,..."
4,4,"{'description': '', 'address1': '475 Park Ave ...",52cdef7c4bab8bd675297daf,"{'price_amount': 62500000, 'price_currency_cod...",[],,,,mobile,[],...,[],"directoryassistance, advertising, mobile",$88.7M,Marchex,Mon Jun 03 05:26:22 UTC 2013,[],88700000.0,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,..."


In [585]:
companies_with_coord_offices_clean_final = companies_with_coord_offices_clean.drop(columns=['index','alias_list','blog_feed_url','blog_url','crunchbase_url','deadpooled_url','external_links','homepage_url','image','offices','permalink','twitter_username','video_embeds'])

In [586]:
companies_with_coord_offices_clean_final = companies_with_coord_offices_clean.drop(columns=['_id'])

In [587]:
companies_with_coord_offices_clean_final.head()

,index,office,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,...,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD,lat,lng,oficina_principal
0,0,"{'description': 'Headquarters', 'address1': '9...","{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,...,[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,1,"{'description': '', 'address1': '100 5th Ave F...","{'price_amount': None, 'price_currency_code': ...",[],None,http://blog.joost.com/atom.xml,http://blog.joost.com,games_video,"[{'competitor': {'name': 'Babelgum', 'permalin...",Sat Jun 09 07:05:30 UTC 2007,...,[],"iptv, babelgum, television, video, thevenicepr...",$45M,None,Fri Mar 15 23:31:11 UTC 2013,"[{'embed_code': '<object width=""425"" height=""3...",45000000.0,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
2,2,"{'description': '', 'address1': '3525 Eastham ...",None,[],,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,"[{'competitor': {'name': 'Topicle', 'permalink...",Thu Jun 14 03:42:20 UTC 2007,...,[],"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",21000000.0,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,3,"{'description': None, 'address1': '442 Post St...","{'price_amount': None, 'price_currency_code': ...",[],,http://feeds.feedburner.com/kyte_blog,http://kyte.com/blog,games_video,"[{'competitor': {'name': 'Ustream', 'permalink...",Thu Jun 14 18:26:11 UTC 2007,...,[],"video, mobile, iphone-app, video-platform, mob...",$23.4M,kyte,Mon Oct 28 09:34:37 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",23400000.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,..."
4,4,"{'description': '', 'address1': '475 Park Ave ...","{'price_amount': 62500000, 'price_currency_cod...",[],,,,mobile,[],Sat Jun 16 05:53:38 UTC 2007,...,[],"directoryassistance, advertising, mobile",$88.7M,Marchex,Mon Jun 03 05:26:22 UTC 2013,[],88700000.0,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,..."


In [588]:
companies_with_coord_offices_clean_final.shape

(2247, 47)

## Filtrando por empresas en las que se ha invertido más de 1.000.000 USD

In [589]:
companies_with_coord_offices_clean_final_2 = companies_with_coord_offices_clean_final[companies_with_coord_offices_clean_final['total_money_raised_normalized_USD']>=1000000][['index', 'office', 'acquisition','category_code', 'deadpooled_year', 'founded_year','funding_rounds', 'investments', 'ipo', 'name', 'number_of_employees', 'products','total_money_raised_normalized_USD', 'lat', 'lng', 'oficina_principal']]
companies_with_coord_offices_clean_final_2.reset_index(inplace=True)

## Exportando a JSON

In [590]:
companies_with_coord_offices_clean_final_2.to_json('data_clean.json', orient="records")

## Exportando a CSV

In [591]:
companies_with_coord_offices_clean_final_2.to_csv('data_clean.csv')

In [592]:
#mongoimport --db companies --collection offices --jsonArray ./data_clean.json

## Geoquery NEAR

In [593]:
def findNear(geopoint, radio_max_meters=1000):
    return db.offices.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 100
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))

Hay 2 oficinas cerca a 100 metros
La mas cercana es
[{'_id': ObjectId('5d2b67203a1bc86dcd57c152'), 'index': 2405, 'office': {'description': 'SpaBooker Head Office', 'address1': '22 Cortlandt St Floor 18', 'address2': '', 'zip_code': '10007', 'city': 'New York', 'state_code': 'NY', 'country_code': 'USA', 'latitude': 40.738567, 'longitude': -73.987199}, 'acquisition': {'price_amount': None, 'price_currency_code': 'USD', 'term_code': 'stock', 'source_url': 'http://www.spabookerblog.com/2010/11/gramercyone-launched-to-grow-booker.html', 'source_description': 'SpaBooker Blog', 'acquired_year': 2010, 'acquired_month': 10, 'acquired_day': 1, 'acquiring_company': {'name': 'Booker', 'permalink': 'booker-software'}}, 'category_code': 'software', 'deadpooled_year': None, 'founded_year': 2007, 'funding_rounds': [{'id': 10391, 'round_code': 'a', 'source_url': '', 'source_description': '', 'raised_amount': 14500000, 'raised_currency_code': 'USD', 'funded_year': 2011, 'funded_month': 10, 'funded_day'

/Users/molins/anaconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [594]:
mapa = folium.Map(location=[40, -45], zoom_start=2.5)
latlng = companies_with_coord_offices_clean_final_2[['lat', 'lng']].values
mapa.add_child(plugins.HeatMap(latlng, radius=20))
mapa

In [595]:
companies_with_coord_offices_clean_final_2.head()

,level_0,index,office,acquisition,category_code,deadpooled_year,founded_year,funding_rounds,investments,ipo,name,number_of_employees,products,total_money_raised_normalized_USD,lat,lng,oficina_principal
0,0,0,"{'description': 'Headquarters', 'address1': '9...","{'price_amount': None, 'price_currency_code': ...",web,None,2006,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],None,Geni,18,"[{'name': 'Geni', 'permalink': 'geni'}]",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,1,1,"{'description': '', 'address1': '100 5th Ave F...","{'price_amount': None, 'price_currency_code': ...",games_video,None,2006,"[{'id': 19, 'round_code': 'a', 'source_url': '...",[],None,Joost,0,"[{'name': 'Joost', 'permalink': 'joost'}]",45000000.0,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,..."
2,2,2,"{'description': '', 'address1': '3525 Eastham ...",None,web,None,2007,"[{'id': 323, 'round_code': 'a', 'source_url': ...",[],None,Mahalo,40,"[{'name': 'Mahalo', 'permalink': 'mahalo'}]",21000000.0,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,..."
3,3,3,"{'description': None, 'address1': '442 Post St...","{'price_amount': None, 'price_currency_code': ...",games_video,None,2006,"[{'id': 32, 'round_code': 'a', 'source_url': '...",[],None,Kyte,40,"[{'name': 'Kyte', 'permalink': 'kyte'}, {'name...",23400000.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,..."
4,4,4,"{'description': '', 'address1': '475 Park Ave ...","{'price_amount': 62500000, 'price_currency_cod...",mobile,None,2005,"[{'id': 36, 'round_code': 'c', 'source_url': '...",[],None,Jingle Networks,35,"[{'name': 'MyFree411', 'permalink': 'myfree411'}]",88700000.0,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,..."


## Cálculo de la densidad monetaria, que la he definido como dinero invertido en las empresas que se encuentran dentro de un radio dividido por el número total de empresas que se encuentran dentro de dicho radio

In [596]:
def findNear(geopoint, radio_max_meters=1000):
    return db.offices.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })

In [597]:
near=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    near.append(findNear(companies_with_coord_offices_clean_final_2['oficina_principal'][i]).count())

#print(companies_with_coord_offices_clean_final_2['oficina_principal'][1])

/Users/molins/anaconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [598]:
near_money=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    x= findNear(companies_with_coord_offices_clean_final_2['oficina_principal'][i])#.count()-1)
    total=0
    for j in range(x.count()):
        total+=x[j]['total_money_raised_normalized_USD']
    near_money.append(total)
#print(companies_with_coord_offices_clean_final_2['oficina_principal'][1])

/Users/molins/anaconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


In [599]:
money_density=[]
for i in range(len(near)):
    money_density.append(near_money[i]/near[i])

In [600]:
companies_with_coord_offices_clean_final_2['money_density']=money_density
companies_with_coord_offices_clean_final_2.head()

,level_0,index,office,acquisition,category_code,deadpooled_year,founded_year,funding_rounds,investments,ipo,name,number_of_employees,products,total_money_raised_normalized_USD,lat,lng,oficina_principal,money_density
0,0,0,"{'description': 'Headquarters', 'address1': '9...","{'price_amount': None, 'price_currency_code': ...",web,None,2006,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",[],None,Geni,18,"[{'name': 'Geni', 'permalink': 'geni'}]",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.650000e+07
1,1,1,"{'description': '', 'address1': '100 5th Ave F...","{'price_amount': None, 'price_currency_code': ...",games_video,None,2006,"[{'id': 19, 'round_code': 'a', 'source_url': '...",[],None,Joost,0,"[{'name': 'Joost', 'permalink': 'joost'}]",45000000.0,40.746497,-74.009447,"{'type': 'Point', 'coordinates': [-74.0094471,...",2.187500e+07
2,2,2,"{'description': '', 'address1': '3525 Eastham ...",None,web,None,2007,"[{'id': 323, 'round_code': 'a', 'source_url': ...",[],None,Mahalo,40,"[{'name': 'Mahalo', 'permalink': 'mahalo'}]",21000000.0,34.017606,-118.487267,"{'type': 'Point', 'coordinates': [-118.487267,...",1.376667e+07
3,3,3,"{'description': None, 'address1': '442 Post St...","{'price_amount': None, 'price_currency_code': ...",games_video,None,2006,"[{'id': 32, 'round_code': 'a', 'source_url': '...",[],None,Kyte,40,"[{'name': 'Kyte', 'permalink': 'kyte'}, {'name...",23400000.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,...",2.861771e+07
4,4,4,"{'description': '', 'address1': '475 Park Ave ...","{'price_amount': 62500000, 'price_currency_cod...",mobile,None,2005,"[{'id': 36, 'round_code': 'c', 'source_url': '...",[],None,Jingle Networks,35,"[{'name': 'MyFree411', 'permalink': 'myfree411'}]",88700000.0,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [-122.173887,...",6.635000e+07


## Ordenando el dataframe por la densidad monetaria

In [601]:
companies_with_coord_offices_clean_final_2 = companies_with_coord_offices_clean_final_2.sort_values(by='money_density',ascending=False).drop(['level_0','index'],axis=1)

In [602]:
companies_with_coord_offices_clean_final_2.reset_index(drop=True, inplace=True)
companies_with_coord_offices_clean_final_2.head()

,office,acquisition,category_code,deadpooled_year,founded_year,funding_rounds,investments,ipo,name,number_of_employees,products,total_money_raised_normalized_USD,lat,lng,oficina_principal,money_density
0,"{'description': '', 'address1': '1051 East Hil...",None,software,None,2007,"[{'id': 1763, 'round_code': 'a', 'source_url':...",[],None,Zuora,250,[],128000000.0,37.559628,-122.270987,"{'type': 'Point', 'coordinates': [-122.2709868...",3.196667e+08
1,"{'description': 'Elevance Renewable', 'address...",None,cleantech,None,2007,"[{'id': 6041, 'round_code': 'a', 'source_url':...",[],None,Elevance Renewable Sciences,68,[],294000000.0,41.675004,-88.064215,"{'type': 'Point', 'coordinates': [-88.0642145,...",2.940000e+08
2,"{'description': None, 'address1': '130 West Un...",None,cleantech,None,2007,"[{'id': 2102, 'round_code': 'b', 'source_url':...",[],None,eSolar,140,[],192000000.0,34.146436,-118.153136,"{'type': 'Point', 'coordinates': [-118.153136,...",1.920000e+08
3,"{'description': 'TestPlant', 'address1': '3000...",None,software,None,2008,"[{'id': 7135, 'round_code': 'unattributed', 's...",[],None,Testplant,50,"[{'name': 'eggPlant', 'permalink': 'eggplant'}]",2560000.0,40.023005,-105.253284,"{'type': 'Point', 'coordinates': [-105.2532841...",1.797800e+08
4,"{'description': '', 'address1': '1500 Fashion ...",None,web,None,2007,"[{'id': 634, 'round_code': 'a', 'source_url': ...",[],None,SpeedDate,0,"[{'name': 'SpeedDate.com', 'permalink': 'speed...",11600000.0,37.557637,-122.285714,"{'type': 'Point', 'coordinates': [-122.285714,...",1.330586e+08


In [603]:
companies_with_coord_offices_clean_final_2['office'][100]

{'description': 'United States',
 'address1': '300 De Haro St',
 'address2': 'South Train',
 'zip_code': '94103',
 'city': 'San Francisco',
 'state_code': 'CA',
 'country_code': 'USA',
 'latitude': 37.787204,
 'longitude': -122.397884}

In [604]:
young_companies=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    x= findNear(companies_with_coord_offices_clean_final_2['oficina_principal'][i])#.count()-1)
    young=0
    for j in range(x.count()):
        if x[j]['founded_year']>=2010:
            young+=1
    young_companies.append(young)
#print(companies_with_coord_offices_clean_final_2['oficina_principal'][1])

/Users/molins/anaconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


In [605]:
young_companies.count(2)

9

In [606]:
companies_with_coord_offices_clean_final_2['young_companies']=young_companies

## Haciendo ranking ponderando con un 75% a la densidad monetaria y con un 25% a la cercanía de empresas jóvenes.

In [607]:
max_comp = companies_with_coord_offices_clean_final_2['young_companies'].max()
max_mon_den = companies_with_coord_offices_clean_final_2['money_density'].max()

In [608]:
ranking=[]
for i in range(len(companies_with_coord_offices_clean_final_2['young_companies'])):
    ranking.append((companies_with_coord_offices_clean_final_2['money_density'][i]/max_mon_den)*0.75+ (companies_with_coord_offices_clean_final_2['young_companies'][i]/max_comp)*0.25)

In [609]:
companies_with_coord_offices_clean_final_2['ranking']=ranking

In [610]:
companies_with_coord_offices_clean_final_2 = companies_with_coord_offices_clean_final_2.sort_values(by=['ranking','money_density','young_companies'],ascending=[False,False,False])

In [611]:
companies_with_coord_offices_clean_final_2 = companies_with_coord_offices_clean_final_2[companies_with_coord_offices_clean_final_2['ranking']>=0.164229].reset_index()
companies_with_coord_offices_clean_final_2.head()

,index,office,acquisition,category_code,deadpooled_year,founded_year,funding_rounds,investments,ipo,name,number_of_employees,products,total_money_raised_normalized_USD,lat,lng,oficina_principal,money_density,young_companies,ranking
0,0,"{'description': '', 'address1': '1051 East Hil...",None,software,None,2007,"[{'id': 1763, 'round_code': 'a', 'source_url':...",[],None,Zuora,250,[],128000000.0,37.559628,-122.270987,"{'type': 'Point', 'coordinates': [-122.2709868...",3.196667e+08,0,0.750000
1,1,"{'description': 'Elevance Renewable', 'address...",None,cleantech,None,2007,"[{'id': 6041, 'round_code': 'a', 'source_url':...",[],None,Elevance Renewable Sciences,68,[],294000000.0,41.675004,-88.064215,"{'type': 'Point', 'coordinates': [-88.0642145,...",2.940000e+08,0,0.689781
2,2,"{'description': None, 'address1': '130 West Un...",None,cleantech,None,2007,"[{'id': 2102, 'round_code': 'b', 'source_url':...",[],None,eSolar,140,[],192000000.0,34.146436,-118.153136,"{'type': 'Point', 'coordinates': [-118.153136,...",1.920000e+08,0,0.450469
3,3,"{'description': 'TestPlant', 'address1': '3000...",None,software,None,2008,"[{'id': 7135, 'round_code': 'unattributed', 's...",[],None,Testplant,50,"[{'name': 'eggPlant', 'permalink': 'eggplant'}]",2560000.0,40.023005,-105.253284,"{'type': 'Point', 'coordinates': [-105.2532841...",1.797800e+08,0,0.421799
4,4,"{'description': '', 'address1': '1500 Fashion ...",None,web,None,2007,"[{'id': 634, 'round_code': 'a', 'source_url': ...",[],None,SpeedDate,0,"[{'name': 'SpeedDate.com', 'permalink': 'speed...",11600000.0,37.557637,-122.285714,"{'type': 'Point', 'coordinates': [-122.285714,...",1.330586e+08,0,0.312181


In [612]:
mapa = folium.Map(location=[40, -45], zoom_start=2.5)
latlng = companies_with_coord_offices_clean_final_2[['lat', 'lng']].values
mapa.add_child(plugins.HeatMap(latlng, radius=20))
mapa

## Usando API Google para encontrar Starbucks en un radio de 100 metros

starbucks=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    lat = companies_with_coord_offices_clean_final_2['lat'][i]
    lng = companies_with_coord_offices_clean_final_2['lng'][i]
    radio = 50
    places=req.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}'.format(str(lat),str(lng),str(radio),'starbucks',api_key)).json()
    starbucks.append(len(places['results']))

In [622]:
companies_with_coord_offices_clean_final_2['starbucks']=starbucks

In [624]:
companies_with_coord_offices_clean_final_2=companies_with_coord_offices_clean_final_2[companies_with_coord_offices_clean_final_2['starbucks']!=0].reset_index(drop=True)

In [646]:
companies_with_coord_offices_clean_final_2.shape

(41, 20)

In [638]:
kinder_garden=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    lat = companies_with_coord_offices_clean_final_2['lat'][i]
    lng = companies_with_coord_offices_clean_final_2['lng'][i]
    radio = 2000
    places=req.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}'.format(str(lat),str(lng),str(radio),'kinder garden',api_key)).json()
    kinder_garden.append(len(places['results']))

schools=[]
for i in range(len(companies_with_coord_offices_clean_final_2)):
    lat = companies_with_coord_offices_clean_final_2['lat'][i]
    lng = companies_with_coord_offices_clean_final_2['lng'][i]
    radio = 2000
    places=req.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}'.format(str(lat),str(lng),str(radio),'school',api_key)).json()
    schools.append(len(places['results']))

In [642]:
companies_with_coord_offices_clean_final_2

,index,office,acquisition,category_code,deadpooled_year,founded_year,funding_rounds,investments,ipo,name,number_of_employees,products,total_money_raised_normalized_USD,lat,lng,oficina_principal,money_density,young_companies,ranking,starbucks
0,0,"{'description': '', 'address1': '1051 East Hil...",None,software,None,2007,"[{'id': 1763, 'round_code': 'a', 'source_url':...",[],None,Zuora,250,[],128000000.0,37.559628,-122.270987,"{'type': 'Point', 'coordinates': [-122.2709868...",3.196667e+08,0,0.750000,1
1,2,"{'description': None, 'address1': '130 West Un...",None,cleantech,None,2007,"[{'id': 2102, 'round_code': 'b', 'source_url':...",[],None,eSolar,140,[],192000000.0,34.146436,-118.153136,"{'type': 'Point', 'coordinates': [-118.153136,...",1.920000e+08,0,0.450469,1
2,3,"{'description': 'TestPlant', 'address1': '3000...",None,software,None,2008,"[{'id': 7135, 'round_code': 'unattributed', 's...",[],None,Testplant,50,"[{'name': 'eggPlant', 'permalink': 'eggplant'}]",2560000.0,40.023005,-105.253284,"{'type': 'Point', 'coordinates': [-105.2532841...",1.797800e+08,0,0.421799,1
3,4,"{'description': '', 'address1': '1500 Fashion ...",None,web,None,2007,"[{'id': 634, 'round_code': 'a', 'source_url': ...",[],None,SpeedDate,0,"[{'name': 'SpeedDate.com', 'permalink': 'speed...",11600000.0,37.557637,-122.285714,"{'type': 'Point', 'coordinates': [-122.285714,...",1.330586e+08,0,0.312181,1
4,141,"{'description': '', 'address1': '300 S Ellswor...",None,advertising,None,2007,"[{'id': 19708, 'round_code': 'b', 'source_url'...",[],None,Inneractive,0,"[{'name': 'InFlare', 'permalink': 'inflare'}]",8500000.0,37.564605,-122.322924,"{'type': 'Point', 'coordinates': [-122.3229235...",2.295000e+07,2,0.303845,1
5,143,"{'description': '', 'address1': '950 Tower Lan...",None,advertising,None,2005,"[{'id': 172, 'round_code': 'a', 'source_url': ...",[],None,Conduit,215,"[{'name': 'Conduit Mobile', 'permalink': 'cond...",110000000.0,37.561877,-122.325248,"{'type': 'Point', 'coordinates': [-122.325248,...",2.295000e+07,2,0.303845,1
6,144,"{'description': '', 'address1': '330 Waverley ...",None,mobile,None,2012,"[{'id': 1760, 'round_code': 'angel', 'source_u...",[],None,Skydeck,9,"[{'name': 'Skydeck', 'permalink': 'skydeck'}]",4000000.0,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2.295000e+07,2,0.303845,1
7,7,"{'description': 'Headquarters', 'address1': '2...",None,advertising,None,2006,"[{'id': 9570, 'round_code': 'c', 'source_url':...",[],"{'valuation_amount': 973500000, 'valuation_cur...",Millennial Media,137,[],240000000.0,39.281734,-76.581864,"{'type': 'Point', 'coordinates': [-76.581864, ...",1.259000e+08,0,0.295386,1
8,395,"{'description': 'Headquarters', 'address1': '1...",None,travel,None,2010,"[{'id': 5948, 'round_code': 'seed', 'source_ur...",[],None,UsingMiles,15,[],3270000.0,39.749671,-104.999845,"{'type': 'Point', 'coordinates': [-104.9998449...",3.270000e+06,2,0.257672,3
9,396,"{'description': 'Headquarters', 'address1': '1...",None,travel,None,2010,"[{'id': 5948, 'round_code': 'seed', 'source_ur...",[],None,UsingMiles,15,[],3270000.0,39.749671,-104.999845,"{'type': 'Point', 'coordinates': [-104.9998449...",3.270000e+06,2,0.257672,3


In [548]:
companies_with_coord_offices_clean_final_2['kinder_garden']=kinder_garden

In [549]:
#companies_with_coord_offices_clean_final_2=companies_with_coord_offices_clean_final_2[companies_with_coord_offices_clean_final_2['kinder_garden']!=0].reset_index(drop=True)

In [550]:
companies_with_coord_offices_clean_final_2.shape

(18, 22)